In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from scipy.ndimage import gaussian_filter
import geopandas as gpd
import os
from tqdm import tqdm
from pyproj import Transformer
import xarray as xr
import warnings
warnings.filterwarnings('ignore')

# ======== Fonctions auxiliaires ========
def idw_interpolation(points, values, grid_points, power=2, smoothing=0, max_distance=None, min_neighbors=3, max_neighbors=12):
    distances = cdist(grid_points, points, metric='euclidean')
    interpolated = np.full(grid_points.shape[0], np.nan)

    for i in range(grid_points.shape[0]):
        dist_to_point = distances[i, :]
        if max_distance is not None:
            valid_mask = dist_to_point <= max_distance
            if not np.any(valid_mask):
                continue
            dist_to_point = dist_to_point[valid_mask]
            point_values = values[valid_mask]
        else:
            point_values = values

        if len(dist_to_point) > max_neighbors:
            nearest_indices = np.argsort(dist_to_point)[:max_neighbors]
            dist_to_point = dist_to_point[nearest_indices]
            point_values = point_values[nearest_indices]

        if len(dist_to_point) < min_neighbors:
            continue

        dist_to_point = dist_to_point + smoothing
        weights = 1.0 / (dist_to_point ** power)
        weights /= np.sum(weights)
        interpolated[i] = np.sum(weights * point_values)

    return interpolated

def adaptive_idw_parameters(points, extent, resolution):
    area = (extent[2] - extent[0]) * (extent[3] - extent[1])
    density = len(points) / (area / 1e6)

    if density > 5:
        power = 1.2
        max_distance = 20000
        min_neighbors = 3
        max_neighbors = 6
    elif density > 2:
        power = 1.0
        max_distance = 30000
        min_neighbors = 3
        max_neighbors = 8
    else:
        power = 0.8
        max_distance = 60000
        min_neighbors = 2
        max_neighbors = 12

    return power, max_distance, min_neighbors, max_neighbors

# ======== Paramètres généraux ========
df_temp = pd.read_csv("C:/temperature_ile_de_france.csv", sep=";")
df_temp["date"] = pd.to_datetime(df_temp['date'])
df_temp = df_temp[df_temp["date"].dt.year.isin([2017, 2018, 2019])]
df_temp = df_temp[(df_temp['temperature'] > -25) & (df_temp['temperature'] < 60)].copy()

transformer = Transformer.from_crs("EPSG:4326", "EPSG:2154", always_xy=True)
df_temp[['x', 'y']] = df_temp.apply(lambda row: pd.Series(transformer.transform(row['longitude'], row['latitude'])), axis=1)

gdf = gpd.read_file("data/shapefile/IDF.shp").to_crs("EPSG:2154")
extent = gdf.total_bounds

res = 50
x_min, y_min, x_max, y_max = extent
x_coords = np.arange(x_min, x_max, res)
y_coords = np.arange(y_min, y_max, res)
grid_x, grid_y = np.meshgrid(x_coords, y_coords)
grid_points = np.column_stack([grid_x.ravel(), grid_y.ravel()])

nodata_value = np.nan
output_dir = "R:/Direction_Data/0_Projets/Projet_CANCAIR/2025_Projet_Loice/temperatures_interpolated_IDW_NetCDF"
os.makedirs(output_dir, exist_ok=True)

# ======== Détection des fichiers déjà générés ========
done_months = set()
for fname in os.listdir(output_dir):
    if fname.startswith("temperature_IDW_") and fname.endswith(".nc"):
        try:
            parts = fname.split("_")
            year = int(parts[2])
            month = int(parts[3][:2])
            done_months.add((year, month))
        except:
            continue

# ======== Traitement ========
df_temp['year'] = df_temp['date'].dt.year
df_temp['month'] = df_temp['date'].dt.month
grouped = df_temp.groupby(['year', 'month'])

print(f"Interpolation IDW : {len(grouped)} groupes (années x mois)")

for (year, month), df_month in grouped:
    if (year, month) in done_months:
        print(f"⏩ {year}-{month:02d} déjà traité, on saute.")
        continue

    unique_dates = sorted(df_month['date'].dt.date.unique())
    print(f"Traitement {year}-{month:02d} avec {len(unique_dates)} jours")

    data_3d = np.full((len(unique_dates), grid_y.shape[0], grid_x.shape[1]), np.nan, dtype=np.float32)

    for i, date in enumerate(tqdm(unique_dates, desc=f"Interpolation {year}-{month:02d}")):
        df_day = df_month[df_month['date'].dt.date == date]
        if len(df_day) < 2:
            continue

        points = df_day[['x', 'y']].values
        values = df_day['temperature'].values

        power, max_distance, min_neighbors, max_neighbors = adaptive_idw_parameters(points, extent, res)

        grid_temp_flat = idw_interpolation(
            points=points,
            values=values,
            grid_points=grid_points,
            power=power,
            smoothing=100.0,
            max_distance=max_distance,
            min_neighbors=min_neighbors,
            max_neighbors=max_neighbors
        )

        grid_temp = grid_temp_flat.reshape(grid_y.shape)

        sigma = 2.0 if len(df_day) > 10 else 1.0
        grid_temp_smooth = gaussian_filter(grid_temp, sigma=sigma, mode='nearest')

        valid_mask = ~np.isnan(grid_temp)
        grid_temp = np.where(valid_mask, grid_temp_smooth, grid_temp)
        grid_temp = np.clip(grid_temp, -25, 60)

        data_3d[i, :, :] = grid_temp

    ds = xr.Dataset(
        {
            "temperature": (["time", "y", "x"], data_3d)
        },
        coords={
            "time": pd.to_datetime(unique_dates),
            "x": x_coords,
            "y": y_coords
        },
        attrs={
            "description": "Températures interpolées par IDW (journalières)",
            "crs": "EPSG:2154",
            "units": "°C",
            "nodata": nodata_value
        }
    )

    ds.x.attrs['units'] = 'meters'
    ds.y.attrs['units'] = 'meters'
    ds.time.attrs['standard_name'] = 'time'
    ds.time.attrs['long_name'] = 'Date'

    out_filename = f"temperature_IDW_{year}_{month:02d}.nc"
    out_path = os.path.join(output_dir, out_filename)
    ds.to_netcdf(out_path, mode='w', format='NETCDF4', encoding={"temperature": {"zlib": True, "complevel": 5}})

    print(f"✅ Fichier NetCDF sauvegardé : {out_path}")

print("✅ Tous les mois non traités ont été générés.")


Interpolation IDW : 36 groupes (années x mois)
⏩ 2017-01 déjà traité, on saute.
⏩ 2017-02 déjà traité, on saute.
⏩ 2017-03 déjà traité, on saute.
⏩ 2017-04 déjà traité, on saute.
⏩ 2017-05 déjà traité, on saute.
⏩ 2017-06 déjà traité, on saute.
⏩ 2017-07 déjà traité, on saute.
⏩ 2017-08 déjà traité, on saute.
⏩ 2017-09 déjà traité, on saute.
⏩ 2017-10 déjà traité, on saute.
⏩ 2017-11 déjà traité, on saute.
⏩ 2017-12 déjà traité, on saute.
⏩ 2018-01 déjà traité, on saute.
⏩ 2018-02 déjà traité, on saute.
⏩ 2018-03 déjà traité, on saute.
⏩ 2018-04 déjà traité, on saute.
⏩ 2018-05 déjà traité, on saute.
⏩ 2018-06 déjà traité, on saute.
⏩ 2018-07 déjà traité, on saute.
⏩ 2018-08 déjà traité, on saute.
⏩ 2018-09 déjà traité, on saute.
⏩ 2018-10 déjà traité, on saute.
⏩ 2018-11 déjà traité, on saute.
⏩ 2018-12 déjà traité, on saute.
⏩ 2019-01 déjà traité, on saute.
Traitement 2019-02 avec 28 jours


Interpolation 2019-02: 100%|██████████| 28/28 [1:25:15<00:00, 182.68s/it]


✅ Fichier NetCDF sauvegardé : R:/Direction_Data/0_Projets/Projet_CANCAIR/2025_Projet_Loice/temperatures_interpolated_IDW_NetCDF\temperature_IDW_2019_02.nc
Traitement 2019-03 avec 31 jours


Interpolation 2019-03: 100%|██████████| 31/31 [2:46:21<00:00, 321.99s/it] 


RuntimeError: NetCDF: HDF error

In [ ]:
# ===== FONCTION BONUS : VALIDATION CROISÉE =====
def cross_validation_idw(df_day, power=2, max_distance=25000, k_folds=5):
    """
    Validation croisée pour évaluer la qualité de l'interpolation IDW
    """
    from sklearn.model_selection import KFold
    
    points = df_day[['x', 'y']].values
    values = df_day['temperature'].values
    
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    errors = []
    
    for train_idx, test_idx in kf.split(points):
        train_points = points[train_idx]
        train_values = values[train_idx]
        test_points = points[test_idx]
        test_values = values[test_idx]
        
        # Interpoler aux points de test
        predicted = idw_interpolation(
            train_points, train_values, test_points,
            power=power, max_distance=max_distance
        )
        
        # Calculer l'erreur
        valid_mask = ~np.isnan(predicted)
        if np.any(valid_mask):
            mae = np.mean(np.abs(predicted[valid_mask] - test_values[valid_mask]))
            errors.append(mae)
    
    return np.mean(errors) if errors else np.nan

In [ ]:
# Exemple d'utilisation de la validation croisée (décommenter pour tester)
test_date = unique_dates[0]
df_test = df[df['date'].dt.date == test_date]
if len(df_test) >= 10:
    mae = cross_validation_idw(df_test)
    print(f"Erreur moyenne absolue (validation croisée) : {mae:.2f}°C")